In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
def getheaders():
    with open('properties.txt', "r") as f:
        properties = f.read().splitlines()
    return properties

In [3]:
def get_letor_row(df, prop_id, srch_id, entry):
    target = prop_id.iloc[entry][0]
    qid = "qid:" + str(srch_id.iloc[entry][0])
    features = []
    for i, feature in enumerate(properties):
        try: 
            value = df.iloc[entry].loc[feature]
        except:
            pass
        else:
            features.append(str(i + 1) + ":" + str(value))
    features = " ".join(features)
    return " ".join([str(target),qid,features])

In [4]:
def bagging(df):   
    groups = df.groupby('srch_id')
    id_list = np.random.choice(df.srch_id.unique(), df.srch_id.unique().shape[0], replace=True)
    all_indices = []
    for group_id in id_list:
        all_indices += list(groups.get_group(group_id).index.values) 
    return df.iloc[np.array(all_indices)]

In [5]:
def open_set(hotel_set_file):
    hotel_set_type = hotel_set_file.split('/')[-1].split('_')[1][:-3]
    hotel_set = pd.read_csv(hotel_set_file)
    hotel_set.drop(hotel_set.select_dtypes(['object']), inplace=True, axis=1)
    if hotel_set_type == 'train':
        hotel_set = bagging(hotel_set).sort_values('srch_id')
    hotel_set_prop_id = hotel_set[['prop_id']]
    hotel_set_srch_id = hotel_set[['srch_id']]
    file = open("../LambdaMart/%s.txt" % hotel_set_type, "w")
    for entry in range(len(hotel_set)):
        get_letor_row(hotel_set, hotel_set_prop_id, hotel_set_srch_id, entry)
        #print(get_letor_row(hotel_set, hotel_set_prop_id, hotel_set_srch_id, entry), file=file)
    file.close()

In [6]:
test = 'complete_files/hoteltest_testing_set.csv'
train = 'test_files/hoteltest_training_set.csv'
properties = getheaders()

In [7]:
open_set(train)

In [ ]:
open_set(test)